# Using custom containers with AI Platform Training

**Learning Objectives:**
1. Learn how to create a train and a validation split with Big Query
1. Learn how to wrap a machine learning model into a Docker container and train in on CAIP
1. Learn how to use the hyperparameter tunning engine on GCP to find the best hyperparameters
1. Learn how to deploy a trained machine learning model GCP as a rest API and query it

In this lab, you develop, package as a docker image, and run on **AI Platform Training** a training application that trains a multi-class classification model that predicts the type of forest cover from cartographic data. The [dataset](../../../datasets/covertype/README.md) used in the lab is based on **Covertype Data Set** from UCI Machine Learning Repository.

The training code uses `scikit-learn` for data pre-processing and modeling. The code has been instrumented using the `hypertune` package so it can be used with **AI Platform** hyperparameter tuning.


In [2]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple



## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name starts with the `hostedkfp-default-` prefix.

In [3]:
!gsutil ls

gs://artifacts.sk-kfp.appspot.com/
gs://automl-ctype/
gs://sk-kfp-kubeflowpipelines-default/
gs://sk-kfp-kubeflowpipelines-eu/
gs://sk-kfp_cloudbuild/


In [4]:
REGION = 'europe-west1'
#ARTIFACT_STORE = 'gs://sk-kfp-kubeflowpipelines-default'
ARTIFACT_STORE = 'gs://sk-kfp-kubeflowpipelines-eu'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.csv')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.csv')

## Explore the Covertype dataset 

In [5]:
%%bigquery
SELECT *
FROM `covertype_dataset.covertype`

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2085,256,18,150,27,738,176,248,208,914,Cache,C2702,5
1,2125,256,20,30,12,871,169,248,215,300,Cache,C2702,2
2,2146,256,34,150,62,1253,122,237,239,511,Cache,C2702,2
3,2186,256,38,210,102,1294,109,232,244,552,Cache,C2702,2
4,2831,256,25,277,183,1706,153,246,225,1485,Commanche,C2705,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3136,254,12,319,60,5734,193,248,193,2467,Rawah,C7746,1
99996,3242,254,12,636,148,3551,193,248,193,2010,Commanche,C7757,0
99997,2071,255,12,234,63,342,192,247,193,247,Cache,C2706,2
99998,3248,255,12,730,113,725,192,247,193,2724,Commanche,C7756,1


## Create training and validation splits

Use BigQuery to sample training and validation splits and save them to GCS storage
### Create a training split

In [6]:
!bq query \
-n 0 \
--destination_table covertype_dataset.training \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (1, 2, 3, 4)' 

Waiting on bqjob_r4b6017076958dfa6_00000172e4ca4995_1 ... (1s) Current status: DONE   


In [7]:
!bq extract \
--destination_format CSV \
covertype_dataset.training \
$TRAINING_FILE_PATH

Waiting on bqjob_r10050ba38f7f6114_00000172e4ccbbd5_1 ... (0s) Current status: DONE   


### Create a validation split

In [8]:
!bq query \
-n 0 \
--destination_table covertype_dataset.validation \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (8)' 

Waiting on bqjob_r7abbf542959a48e5_00000172e4cd679b_1 ... (1s) Current status: DONE   


In [9]:
!bq extract \
--destination_format CSV \
covertype_dataset.validation \
$VALIDATION_FILE_PATH

Waiting on bqjob_r12eb936ddbcd847d_00000172e4cd973a_1 ... (0s) Current status: DONE   


##### Read directly from cloud storage to pandas dataframe

In [10]:
TRAINING_FILE_PATH

'gs://sk-kfp-kubeflowpipelines-eu/data/training/dataset.csv'

In [11]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

(40009, 13)
(9836, 13)


## Develop a training application

### Run the pipeline locally.

In [17]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  slice(0, 10, None)),
                                                 ('cat', OneHotEncoder(),
                                                  slice(10, 12, None))])),
                ('classifier',
                 SGDClassifier(alpha=0.001, loss='log', max_iter=200))])

### Calculate the trained model's accuracy.

In [18]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.6940829605530704


### Prepare the hyperparameter tuning application.
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

In [19]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

### Write the tuning script. 

Notice the use of the `hypertune` package to report the `accuracy` optimization metric to AI Platform hyperparameter tuning service.

In [20]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    numeric_feature_indexes = slice(0, 10)
    categorical_feature_indexes = slice(10, 12)

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier(loss='log',tol=1e-3))
    ])

    num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map) 

    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    X_train = df_train.drop('Cover_Type', axis=1)
    y_train = df_train['Cover_Type']

    pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df_validation.drop('Cover_Type', axis=1)
        y_validation = df_validation['Cover_Type']
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
    fire.Fire(train_evaluate)

Writing training_app/train.py


In [1]:
%%writefile {TRAINING_APP_FOLDER}/train_fastai.py


from fastai_custom.imports import *
from fastai_custom.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn.metrics import accuracy_score

from sklearn import metrics


import os
PATH=os.path.join(os.getcwd(),'data')

df_train = pd.read_csv(f'{PATH}/training/dataset.csv', low_memory=False)

train_cats(df_train)

df_valid = pd.read_csv(f'{PATH}/validation/dataset.csv', low_memory=False)

df_raw = pd.concat([df_train,df_valid], axis=0, ignore_index=True)

df, y, nas = proc_df(df_raw, 'Cover_Type')

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 9836  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

m = RandomForestClassifier(n_estimators=200, n_jobs=-1,oob_score=True,min_samples_leaf=1,max_features='sqrt')
m.fit(X_train, y_train)
print_score(m)

t=m.estimators_[0].tree_
dectree_max_depth(t)

feat_imp = m.feature_importances_

features = X_valid.columns.values

feat_imp_list = [(item[0],round(item[1]*100,2)) for item in list(zip(features, feat_imp))]

feat_imp_list.sort(key=lambda x: x[1], reverse=True)

print(feat_imp_list)


Writing {TRAINING_APP_FOLDER}/train_fastai.py


FileNotFoundError: [Errno 2] No such file or directory: '{TRAINING_APP_FOLDER}/train_fastai.py'

### Package the script into a docker image.

Notice that we are installing specific versions of `scikit-learn` and `pandas` in the training image. This is done to make sure that the training runtime is aligned with the serving runtime. Later in the notebook you will deploy the model to AI Platform Prediction, using the 1.15 version of AI Platform Prediction runtime. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [21]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Writing training_app/Dockerfile


### Build the docker image. 

You use **Cloud Build** to build the image and push it your project's **Container Registry**. As you use the remote cloud service to build the image, you don't need a local installation of Docker.

In [22]:
PROJECT_ID

'sk-kfp'

In [23]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [24]:
IMAGE_URI

'gcr.io/sk-kfp/trainer_image:latest'

In [25]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 3 file(s) totalling 6.2 KiB before compression.
Uploading tarball of [training_app] to [gs://sk-kfp_cloudbuild/source/1592576587.14-8296af658c01417baa6118fc33e215e2.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/sk-kfp/builds/41abf9b8-2429-4ea2-a897-8555540deadd].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/41abf9b8-2429-4ea2-a897-8555540deadd?project=1069413358010].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "41abf9b8-2429-4ea2-a897-8555540deadd"

FETCHSOURCE
Fetching storage object: gs://sk-kfp_cloudbuild/source/1592576587.14-8296af658c01417baa6118fc33e215e2.tgz#1592576587824763
Copying gs://sk-kfp_cloudbuild/source/1592576587.14-8296af658c01417baa6118fc33e215e2.tgz#1592576587824763...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 objects/1.6 KiB.                                      
BUILD

## Submit an AI Platform hyperparameter tuning job

### Create the hyperparameter configuration file. 
Recall that the training code uses `SGDClassifier`. The training application has been designed to accept two hyperparameters that control `SGDClassifier`:
- Max iterations
- Alpha

The below file configures AI Platform hypertuning to run up to 6 trials on up to three nodes and to choose from two discrete values of `max_iter` and the linear range betwee 0.00001 and 0.001 for `alpha`.

In [26]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.001
      scaleType: UNIT_LINEAR_SCALE

Writing training_app/hptuning_config.yaml


### Start the hyperparameter tuning job.

Use the `gcloud` command to start the hyperparameter tuning job.

In [28]:
IMAGE_URI

'gcr.io/sk-kfp/trainer_image:latest'

In [27]:
JOB_DIR_ROOT

'gs://sk-kfp-kubeflowpipelines-eu/jobs'

In [29]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune

Job [JOB_20200619_143537] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200619_143537

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200619_143537
jobId: JOB_20200619_143537
state: QUEUED


### Monitor the job.

You can monitor the job using GCP console or from within the notebook using `gcloud` commands.

In [33]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2020-06-19T14:35:40Z'
endTime: '2020-06-19T14:44:05Z'
etag: p4yFLezacyw=
jobId: JOB_20200619_143537
startTime: '2020-06-19T14:35:42Z'
state: SUCCEEDED
trainingInput:
  args:
  - --training_dataset_path=gs://sk-kfp-kubeflowpipelines-eu/data/training/dataset.csv
  - --validation_dataset_path=gs://sk-kfp-kubeflowpipelines-eu/data/validation/dataset.csv
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 0.001
      minValue: 1e-05
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://sk-kfp-kubeflowpipelines-eu/jobs/JOB_20200619_143537
  masterConfig:
    imageUri: gcr.io/sk-kfp/trainer_image:latest
  region: europe-west1
trainingOutput:
  completedTrialCount: '4'
  consumedMLUnits: 0.27
  hyperp

In [34]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

^C


Command killed by keyboard interrupt



### Retrieve HP-tuning results.

After the job completes you can review the results using GCP Console or programatically by calling the AI Platform Training REST end-point.

In [35]:
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response

{'jobId': 'JOB_20200619_143537',
 'trainingInput': {'args': ['--training_dataset_path=gs://sk-kfp-kubeflowpipelines-eu/data/training/dataset.csv',
   '--validation_dataset_path=gs://sk-kfp-kubeflowpipelines-eu/data/validation/dataset.csv',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'max_iter',
     'type': 'DISCRETE',
     'discreteValues': [200, 500]},
    {'parameterName': 'alpha',
     'minValue': 1e-05,
     'maxValue': 0.001,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'}],
   'maxTrials': 4,
   'maxParallelTrials': 4,
   'hyperparameterMetricTag': 'accuracy',
   'enableTrialEarlyStopping': True},
  'region': 'europe-west1',
  'jobDir': 'gs://sk-kfp-kubeflowpipelines-eu/jobs/JOB_20200619_143537',
  'masterConfig': {'imageUri': 'gcr.io/sk-kfp/trainer_image:latest'}},
 'createTime': '2020-06-19T14:35:40Z',
 'startTime': '2020-06-19T14:35:42Z',
 'endTime': '2020-06-19T14:44:05Z',
 'state': 'SUCCEEDED',
 'trainingOutput': {

The returned run results are sorted by a value of the optimization metric. The best run is the first item on the returned list.

In [36]:
response['trainingOutput']['trials'][0]

{'trialId': '3',
 'hyperparameters': {'alpha': '0.00027792392060024715', 'max_iter': '200'},
 'finalMetric': {'trainingStep': '1', 'objectiveValue': 0.7072997153314355},
 'startTime': '2020-06-19T14:36:20.457938191Z',
 'endTime': '2020-06-19T14:43:20Z',
 'state': 'SUCCEEDED'}

## Retrain the model with the best hyperparameters

You can now retrain the model using the best hyperparameters and using combined training and validation splits as a training dataset.

### Configure and run the training job

In [37]:
alpha = response['trainingOutput']['trials'][0]['hyperparameters']['alpha']
max_iter = response['trainingOutput']['trials'][0]['hyperparameters']['max_iter']

In [38]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--alpha=$alpha \
--max_iter=$max_iter \
--nohptune

Job [JOB_20200619_144940] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200619_144940

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200619_144940
jobId: JOB_20200619_144940
state: QUEUED


In [ ]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

### Examine the training output

The training script saved the trained model as the 'model.pkl' in the `JOB_DIR` folder on GCS.

In [39]:
!gsutil ls $JOB_DIR

gs://sk-kfp-kubeflowpipelines-eu/jobs/JOB_20200619_144940/model.pkl


## Deploy the model to AI Platform Prediction

### Create a model resource

In [74]:
model_name = 'forest_cover_classifier'
labels = "task=classifier,domain=forestry"
filter = 'name:{}'.format(model_name)
models = !(gcloud ai-platform models  -q list --filter={filter} --format='value(name)')

if not models or (models and ("WARNING" in models[0])) :
    print("creating model..")
    !gcloud ai-platform models create  $model_name \
    --regions=$REGION \
    --labels=$labels
else:
    print("Model: {} already exists.".format(models[0]))

creating model..
Created ml engine model [projects/sk-kfp/models/forest_cover_classifier].


### Create a model version

In [75]:
'WARNING' in models[0]

True

In [76]:
!gcloud ai-platform models list

NAME                     DEFAULT_VERSION_NAME
forest_cover_classifier


In [41]:
JOB_DIR

'gs://sk-kfp-kubeflowpipelines-eu/jobs/JOB_20200619_144940'

In [80]:
model_version = 'v01'
filter = 'name:{}'.format(model_version)
versions = !(gcloud ai-platform versions list --model={model_name} --format='value(name)' --filter={filter})

if not versions or (versions and "WARNING" in versions[0]):
    !gcloud ai-platform versions create {model_version} \
    --model={model_name} \
    --origin=$JOB_DIR \
    --runtime-version=1.15 \
    --framework=scikit-learn \
    --python-version=3.7
else:
    print("Model version: {} already exists.".format(versions[0]))

ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


In [83]:
versions[1]

'v01'

### Serve predictions
#### Prepare the input file with JSON formated instances.

In [88]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.tail(50).iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [89]:
!cat $input_file

[1979.0, 4.0, 38.0, 30.0, 19.0, 90.0, 146.0, 142.0, 113.0, 212.0, "Cache", "C4703"]
[2116.0, 306.0, 38.0, 30.0, 26.0, 182.0, 92.0, 189.0, 222.0, 607.0, "Cache", "C4703"]
[2192.0, 346.0, 38.0, 510.0, 35.0, 644.0, 123.0, 153.0, 151.0, 914.0, "Cache", "C4703"]
[2659.0, 161.0, 38.0, 85.0, 47.0, 1440.0, 226.0, 217.0, 85.0, 1026.0, "Commanche", "C4758"]
[2821.0, 244.0, 38.0, 190.0, 112.0, 2290.0, 118.0, 237.0, 236.0, 878.0, "Commanche", "C4758"]
[2782.0, 192.0, 38.0, 108.0, 24.0, 1310.0, 189.0, 237.0, 150.0, 1959.0, "Commanche", "C4758"]
[2771.0, 219.0, 38.0, 127.0, 82.0, 1950.0, 150.0, 243.0, 203.0, 1120.0, "Commanche", "C4758"]
[2686.0, 86.0, 38.0, 120.0, 55.0, 2731.0, 243.0, 149.0, 0.0, 6570.0, "Rawah", "C7746"]
[3012.0, 289.0, 38.0, 379.0, 186.0, 1353.0, 92.0, 205.0, 239.0, 1100.0, "Commanche", "C7757"]
[2963.0, 329.0, 38.0, 484.0, 240.0, 2839.0, 105.0, 167.0, 186.0, 376.0, "Commanche", "C7757"]
[2980.0, 325.0, 38.0, 350.0, 207.0, 819.0, 103.0, 171.0, 193.0, 1302.0, "Commanche", "C7757"]

#### Invoke the model

In [90]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file

[5, 5, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 6, 1, 1, 0, 0, 0, 0, 6, 6, 2, 2, 0, 0, 6, 0, 0, 6, 2, 2, 1, 0, 2, 2, 0, 2, 2, 5, 0, 0, 2, 2, 4, 0, 2, 5, 0, 1]


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>